In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline
import gym
import collections
import random

In [33]:
# 하이퍼파라미터

lr = 0.0005
gamma = 0.99 # 감마의 경우 리턴을 계산할 때 사용
buffer_limit = 50000 # 리플레이버퍼의 길이
batch_size = 32 # 학습시 사용되는 transition의 사이즈

In [34]:
# Q network : 상태를 입력받아서 행동을 출력

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4,128) # 상태 4개
        self.fc2 = nn.Linear(128,2) # 액션 2개
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        
    def sample_action(self, obs, eps):
        out = self.forward(obs)
        coin = random.random()
        if coin < eps:
            return random.randint(0, 1)
        else:
            return out.argmax().item()

In [35]:
class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        # 리플레이 버퍼에 데이터를 넣는다.
        self.buffer.append(transition)
    
    def size(self):
        return len(self.buffer)
        
    def sample(self, n):
        # q-net을 학습할 때 배치 데이터를 추출한다.
        m_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            # 버퍼에서 배치를 추출한 다음 파이토치 텐서 오브젝트로 리턴한다.
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)

In [36]:
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s, a, r, s,_prime, done_mask = memory.sample(batch_size)
        q_out = q(s)
        # gather() 다중 인덱싱 기능 : 행렬에서 특정 원소들을 뽑아낸다.
        q_a = q_out.gather(1, a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma + max_q_prime * done_mask
        loss = F.mse_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [37]:
def main():
    # gym.make('환경이름') 메소드는 환경 인스턴스를 반환
    env = gym.make('CartPole-v1')
    
    # q, q_target을 기존에 구현한 Qnet()클래스의 인스턴스로 할당
    q = Qnet()
    q_target = Qnet()
    
    # Load_state_dict를 통해서 q의 가중치 정보를 q_target으로 복사
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()

    print_interval = 20
    score = 0.0  
    
    # 옵티마이저 정의
    optimizer = optim.Adam(q.parameters(), lr=lr)

    # 환경을 10000번 에피소드만큼 반복
    for n_epi in range(10000):
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
        s = env.reset()
        done = False

        # done 변수는 환경이 끝났음을 알려준다
        while not done:
            env.render()
            
            # A : 엡실론 그리디를 통해 행동 선택
            a = q.sample_action(torch.from_numpy(s).float(), epsilon)    
            
            # B : env.step(action)은 행동을 입력받아 다음 상태, 보상, done 정보를 반환  
            s_prime, r, done, info = env.step(a)
            done_mask = 0.0 if done else 1.0
            
            # F : transaction을 리플레이버퍼에 저장
            memory.put((s,a,r/100.0,s_prime, done_mask))
            
            # E : 상태를 업데이트
            s = s_prime

            score += r
            if done:
                break
            
        if memory.size()>2000:
            
            # D : 모델 파라미터를 업데이트
            train(q, q_target, memory, optimizer)

        if n_epi%print_interval==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0
    env.close()

if __name__ == '__main__':
    main()

NoSuchDisplayException: Cannot connect to "None"

In [15]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
buffer_limit  = 50000
batch_size    = 32

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,1)
        else : 
            return out.argmax().item()
            
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def main():
    env = gym.make('CartPole-v1')
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()

    print_interval = 20
    score = 0.0  
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)

    for n_epi in range(10000):
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
        s = env.reset()
        done = False

        while not done:
            a = q.sample_action(torch.from_numpy(s).float(), epsilon)      
            s_prime, r, done, info = env.step(a)
            done_mask = 0.0 if done else 1.0
            memory.put((s,a,r/100.0,s_prime, done_mask))
            s = s_prime

            score += r
            if done:
                break
            
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)

        if n_epi%print_interval==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0
    env.close()

if __name__ == '__main__':
    main()

n_episode :20, score : 9.9, n_buffer : 198, eps : 7.9%
n_episode :40, score : 9.8, n_buffer : 393, eps : 7.8%
n_episode :60, score : 9.7, n_buffer : 586, eps : 7.7%
n_episode :80, score : 9.7, n_buffer : 779, eps : 7.6%
n_episode :100, score : 10.4, n_buffer : 987, eps : 7.5%
n_episode :120, score : 9.8, n_buffer : 1182, eps : 7.4%
n_episode :140, score : 9.6, n_buffer : 1373, eps : 7.3%
n_episode :160, score : 9.7, n_buffer : 1566, eps : 7.2%
n_episode :180, score : 9.4, n_buffer : 1754, eps : 7.1%
n_episode :200, score : 9.7, n_buffer : 1947, eps : 7.0%


/home/chromis/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:35: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


n_episode :220, score : 14.1, n_buffer : 2229, eps : 6.9%
n_episode :240, score : 10.1, n_buffer : 2431, eps : 6.8%
n_episode :260, score : 10.1, n_buffer : 2632, eps : 6.7%
n_episode :280, score : 10.3, n_buffer : 2838, eps : 6.6%
n_episode :300, score : 10.4, n_buffer : 3047, eps : 6.5%
n_episode :320, score : 18.5, n_buffer : 3417, eps : 6.4%
n_episode :340, score : 54.7, n_buffer : 4511, eps : 6.3%
n_episode :360, score : 122.2, n_buffer : 6956, eps : 6.2%


KeyboardInterrupt: 